In [1]:
import sys
project_root_path = '../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
import commit_stats

In [2]:
import json
in_path = project_root_path + '/python/cfg/ds_pipeline.json'
config_map = json.loads(open(in_path, 'r').read())
config_map

cfg = config_map['matt_language_analyzer'] # 99_problems

In [3]:

path = project_root_path + '/' + cfg['stats_dir']
all_commit_stats = commit_stats.load_commit_stats(cfg['stat_file_name'], path)
all_commit_stats[slice(0,2)]

[{'commit': 'd91d238c18dd8794a1031ed2f66d0a1591f88212',
  'Author': 'Bacon Huang <bacon_huang@migocorp.com>',
  'Date': '2020-08-26T11:11:48+00:00',
  'orig_timezone': 'UTC+08:00',
  'fileTypes': {'py': {'inserts': 4, 'deletes': 3, 'occurrences': 1}},
  'file_list': {'sklearn/ensemble/_forest.py\n': {'binary': 0,
    'inserts': 4,
    'deletes': 3}},
  'seen_in': ['../active'],
  'author_hash': '67657c47c7f9e3f85e81a33947f91931',
  'iso_datetime': '2020-08-26T11:11:48+00:00',
  'utc_datetime': '2020-08-26T11:11:48'},
 {'commit': '8ba49f628092fe3fb1deea101910006aa6c76d49',
  'Author': 'Venkatachalam N <venky.yuvy@gmail.com>',
  'Date': '2020-08-26T09:10:24+00:00',
  'orig_timezone': 'UTC+05:30',
  'fileTypes': {'rst': {'inserts': 13, 'deletes': 0, 'occurrences': 1},
   'py': {'inserts': 101, 'deletes': 12, 'occurrences': 6}},
  'file_list': {'doc/whats_new/v0.24.rst\n': {'binary': 0,
    'inserts': 13,
    'deletes': 0},
   'sklearn/ensemble/tests/test_common.py\n': {'binary': 0,
    'i

In [4]:
ft_dicts = []
ft_dict_dict = {}
for stats in all_commit_stats:
    for file_type in stats['fileTypes']:
        commit_ft_dict = stats['fileTypes'][file_type]
        if file_type not in ft_dict_dict:
            ft_dict = {}
            ft_dicts.append(ft_dict)
            ft_dict_dict[file_type] = ft_dict
            ft_dict['type'] = file_type
            ft_dict['lines'] = 0
            ft_dict['files'] = 0
            ft_dict['commits'] = 0
            ft_dict['repos'] = {}
        ft_dict = ft_dict_dict[file_type]
        ft_dict['commits'] += 1
        ft_dict['lines'] += commit_ft_dict['inserts']
        ft_dict['files'] += commit_ft_dict['occurrences']
        #seen_in = commit_ft_dict['seen_in']
        #for project in seen_in:
        #    ft_dict['repos'][project] = 1


In [5]:
by_commit_count = sorted(ft_dicts, key=lambda d: d['commits'])
by_commit_count.reverse()
# if your odds of being in a commit are < 1/1000, you don't matter
main_type_stats = [stats for stats in by_commit_count if stats['commits'] > (len(all_commit_stats) / 1000)]
print(len(main_type_stats))
main_type_stats

45


[{'type': 'py',
  'lines': 86964899,
  'files': 3453687,
  'commits': 1378129,
  'repos': {}},
 {'type': 'rst',
  'lines': 9566242,
  'files': 451594,
  'commits': 283764,
  'repos': {}},
 {'type': 'noexttext',
  'lines': 2949579,
  'files': 231112,
  'commits': 119625,
  'repos': {}},
 {'type': 'c',
  'lines': 97977422,
  'files': 244157,
  'commits': 107309,
  'repos': {}},
 {'type': 'ts',
  'lines': 18043586,
  'files': 436344,
  'commits': 105336,
  'repos': {}},
 {'type': 'js',
  'lines': 18375143,
  'files': 416885,
  'commits': 100878,
  'repos': {}},
 {'type': 'json',
  'lines': 29318839,
  'files': 228660,
  'commits': 84893,
  'repos': {}},
 {'type': 'yml',
  'lines': 1574446,
  'files': 107468,
  'commits': 83802,
  'repos': {}},
 {'type': 'md',
  'lines': 3479046,
  'files': 176274,
  'commits': 80210,
  'repos': {}},
 {'type': 'txt',
  'lines': 4629208,
  'files': 136620,
  'commits': 78553,
  'repos': {}},
 {'type': 'pyx',
  'lines': 4347258,
  'files': 104400,
  'commits

In [ ]:
import pandas as pd

main_types = [stat['type'] for stat in main_type_stats]
main_types.index('json')
rows = []
for commit_stats in all_commit_stats:
    row = [0]*len(main_types)
    rows.append(row)
    for file_type in commit_stats['fileTypes']:
        commit_ft_dict = commit_stats['fileTypes'][file_type]
        if file_type in main_types:
            position = main_types.index(file_type)
            row[position] = commit_ft_dict['inserts']
type_df = pd.DataFrame(rows, columns=main_types) # index=commit_ids,
type_df

In [ ]:
type_corr = type_df.corr()
print(str(type_corr))

In [ ]:
for i in range(len(main_types)):
    for j in range(len(main_types)):
        if i < j:
            val = type_corr.iat[i, j]
            if val > 0.2:
                print(main_types[i] + ", " + main_types[j] + ": " + str(val))